In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [14]:
#Drop  non-beneficial columns
application_df = application_df.drop(columns=['EIN', 'NAME'])


In [16]:
# Analyze category variables
app_counts = application_df['APPLICATION_TYPE'].value_counts()
rare_apps = app_counts[app_counts < 500].index
application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].apply(lambda x: 'Other' if x in rare_apps else x)


In [18]:
application_df = pd.get_dummies(application_df)


In [20]:
# Split data
X = application_df.drop(columns=['IS_SUCCESSFUL'])
y = application_df['IS_SUCCESSFUL']



In [21]:
#Split into training and testings sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [22]:
#Scale Data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [23]:
#Define and Compile Model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(80, activation='relu', input_dim=X_train_scaled.shape[1]),
    tf.keras.layers.Dense(30, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])


c:\Users\Logan\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [24]:
#Compile the Model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [25]:
#Callbacks to save the model
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath='model_weights.weights.h5',
    save_weights_only=True,
    save_freq=5 * len(X_train_scaled)
)


In [26]:
#Train the Model
history = model.fit(X_train_scaled, y_train, epochs=100, batch_size=32, validation_data=(X_test_scaled, y_test), callbacks=[checkpoint_cb])


Epoch 1/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.6903 - loss: 0.6045 - val_accuracy: 0.7286 - val_loss: 0.5612
Epoch 2/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7286 - loss: 0.5542 - val_accuracy: 0.7259 - val_loss: 0.5613
Epoch 3/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7374 - loss: 0.5447 - val_accuracy: 0.7286 - val_loss: 0.5578
Epoch 4/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7343 - loss: 0.5479 - val_accuracy: 0.7255 - val_loss: 0.5570
Epoch 5/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7365 - loss: 0.5416 - val_accuracy: 0.7236 - val_loss: 0.5582
Epoch 6/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7330 - loss: 0.5427 - val_accuracy: 0.7293 - val_loss: 0.5553
Epoch 7/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7355 - loss: 0.5414 - val_accuracy: 0.7297 - val_loss: 0.5545
Epoch 8/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7396 - loss: 0.5373 - val_accu

In [27]:
#Evaluate and Save the Model
loss, accuracy = model.evaluate(X_test_scaled, y_test)
print(f'Loss: {loss}, Accuracy: {accuracy}')


215/215 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7270 - loss: 0.5615
Loss: 0.5670921206474304, Accuracy: 0.7290087342262268


In [28]:
model.save('AlphabetSoupCharity_Optimization.h5')
